In [46]:
# cfgs_all_N

import os,pickle
import h5py
import numpy as np

ens='cE211.044.112'

cfg2old=lambda cfg: cfg[1:]+'_r'+{'a':'0','b':'1','c':'2','d':'3'}[cfg[0]]
cfg2new=lambda cfg: {'0':'a','1':'b','2':'c','3':'d'}[cfg[-1]] + cfg[:4]

path='data_aux/cfgs_all_N'
with open(path,'r') as f:
    cfgs_N=f.read().splitlines()

path='/p/project1/ngff/li47/code/projectData/05_moments/cE211.044.112/data_N_fullmom/'

cfgs=[]
for icfg,cfg in enumerate(cfgs_N):
    flag=True
    for irun,Nsrc in enumerate([100,127,84,16,12],start=1):
        print(icfg,irun,end='            \r')
        file=f'{path}{cfg}/N.h5_Nrun{irun}_{Nsrc}'
        with h5py.File(file) as f:
            Nsrc_this=len(f['srcs'])
            if Nsrc!=Nsrc_this:
                flag=False
                print(cfg,irun,Nsrc,Nsrc_this)
    if cfg in ['a1084','a1236','b0672','b0832','c1432','c1872','d0352']: # nan observed; for b0832, nan appears in middle timeslice not in use
        flag=False
    if flag:
        cfgs.append(cfg)
            
with open('data_aux/cfgs_N=100,127,84,16,12','w') as f:
    f.write('\n'.join(cfgs))

c1320 2 127 95   
c1344 2 127 102  
c1360 2 127 101  
c1368 2 127 126  
c1384 2 127 53   
c1408 2 127 100  
c1440 2 127 107  


In [17]:
# cfgs_all_jlsc

import os,pickle
import h5py
import numpy as np

path='data_aux/cfgs_all_j'
with open(path,'r') as f:
    cfgs_j=f.read().splitlines()

path='/p/project1/ngff/li47/code/projectData/05_moments/cE211.044.112/data_post/'
cfgs=os.listdir(path); cfgs.sort()
cfgs_jl=[]; cfgs_js=[]; cfgs_jc=[]
for i,cfg in enumerate(cfgs):
    print(f'{i}/{len(cfgs)}',end='              \r')
    file=f'{path}{cfg}/j.h5'
    with h5py.File(file) as f:
        if 'j+' in f['data']:
            cfgs_jl.append(cfg)
        if 'js' in f['data']:
            cfgs_js.append(cfg)
        if 'jc' in f['data']:
            cfgs_jc.append(cfg)
    # break

print('all',len(cfgs_j))
print('jl',len(cfgs_jl),set(cfgs_j)-set(cfgs_jl))
print('js',len(cfgs_js),set(cfgs_j)-set(cfgs_js))
print('jc',len(cfgs_jc),set(cfgs_j)-set(cfgs_jc))
cfgs_jlsc=set(cfgs_jl).intersection(set(cfgs_js)).intersection(set(cfgs_jc))
print('jlsc',len(cfgs_jlsc),set(cfgs_j)-set(cfgs_jlsc))

with open('data_aux/cfgs_jlsc','w') as f:
    f.write('\n'.join(cfgs_jlsc))

all 516              
jl 475 {'d0760', 'c0808', 'b0720', 'c0760', 'a0724', 'a0804', 'a0788', 'b0528', 'b0816', 'c0768', 'c0792', 'b0784', 'a0780', 'd0816', 'a0812', 'b0736', 'a0668', 'b0744', 'c0720', 'b0600', 'c0752', 'c0744', 'd0792', 'c0824', 'c0608', 'c0776', 'a0580', 'a0820', 'c0736', 'b0824', 'b0664', 'c0712', 'd0752', 'b0800', 'a0740', 'b0776', 'a0644', 'd0704', 'b0728', 'a0772', 'c1448'}
js 516 set()
jc 515 {'d0560'}
jlsc 474 {'d0760', 'c0808', 'b0720', 'c0760', 'd0560', 'a0724', 'a0804', 'a0788', 'b0528', 'b0816', 'c0768', 'c0792', 'b0784', 'a0780', 'd0816', 'a0812', 'b0736', 'a0668', 'b0744', 'c0720', 'b0600', 'c0752', 'c0744', 'd0792', 'c0824', 'c0608', 'c0776', 'a0580', 'a0820', 'c0736', 'b0824', 'b0664', 'c0712', 'd0752', 'b0800', 'a0740', 'b0776', 'a0644', 'd0704', 'b0728', 'a0772', 'c1448'}


# old

In [ ]:
# collect data from Bhavna

import h5py
import numpy as np

cfg2old=lambda cfg: cfg[1:]+'_r'+{'a':'0','b':'1','c':'2','d':'3'}[cfg[0]]
cfg2new=lambda cfg: {'0':'a','1':'b','2':'c','3':'d'}[cfg[-1]] + cfg[:4]

path='/p/scratch/renormparton/prasad2/disconnected_boosted/E/twop/nucl-twop-avesrc-updated.h5'
with h5py.File(path) as f:
    cfgs=[cfg2new(key) for key in f.keys()]; cfgs.sort()
    cfgs=cfgs[:]
    
    t=np.array([[[[f[cfg2old(cfg)][nucl]['msq000']['P0'][fwd]][:] for fwd in ['fwd','bwd']] for nucl in ['nucl1','nucl2']] for cfg in cfgs])
    t=(t[:,:,0]-t[:,:,1])/2
    t=np.mean(t,axis=1)
    t=t[:,0,:,0]
    
with h5py.File('/p/project1/ngff/li47/code/projectData/05_moments/cE211.044.112/data_merge/disc_2pt.h5','w') as f:
    f.create_dataset('cfgs',data=cfgs)
    f.create_dataset('moms',data=[[0,0,0]])
    f.create_dataset('data/N_N',data=t[:,:,None])
